In [1]:
import os
import re
import json
import torch
import random
import codecs
import gensim
import torchtext
import numpy as np
import pandas as pd

from torch.utils.data import DataLoader

print(f"PyTorch version: {torch.__version__}\nTorchtext version: {torchtext.__version__}")

ImportError: cannot import name 'open' from 'smart_open' (C:\Users\green198800\Anaconda3\lib\site-packages\smart_open\__init__.py)

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backend.cudnn.deterministic = True

In [32]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

In [35]:
def build_data(data_path, cv=10):
  
    revs = []
    pos = codecs.open(os.path.join(data_path, "rt-polarity.pos"), "r", encoding='utf-8', errors='ignore').read()
    neg = codecs.open(os.path.join(data_path, "rt-polarity.neg"), "r", encoding='utf-8', errors='ignore').read()
    pos_list = [clean_str(sent) for sent in pos.split('\n')[:-1]]
    neg_list = [clean_str(sent) for sent in neg.split('\n')[:-1]]
    print('pos len', len(pos_list))
    print('neg len', len(neg_list))
    print('total len',len(pos_list) + len(neg_list) )
  
    for sent in pos_list:
        datum = {'label': 1,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                 }
        revs.append(datum)

    for sent in neg_list:
        datum = {'label': 0,
                 'text': sent,
                 'num_words': len(sent.split()),
                 'split': np.random.randint(0,cv)
                }
        revs.append(datum)

    word_to_idx = {'@pad': 0}

    for sent in pos_list + neg_list:
        for word  in sent.split():
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)

    print(f'num of total data: {len(revs)}')
    print(f'num of vocab: {len(word_to_idx)}')

    df = pd.DataFrame(revs) 
    df.to_csv(os.path.join(data_path, 'polarity_df.csv'), index=False, encoding='utf-8')
    
    print('save the data')
    
    return revs, word_to_idx

In [36]:
data_path = "data"
revs, word_to_idx = build_data(data_path)

pos len 5331
neg len 5331
total len 10662
num of total data: 10662
num of vocab: 18778
save the data


In [38]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT   = torchtext.data.Field(sequential=True, tokenize=str.split, batch_first=True, fix_length=50, lower=True)
LABEL  = torchtext.data.Field(sequential=False, dtype=torch.float)
FIELDS = [('text', TEXT),('label', LABEL)]

dataset = torchtext.data.TabularDataset(os.path.join(data_path, "polarity_df.csv"), fields=FIELDS, format='csv')

C:\Users\green198800\Anaconda3\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
C:\Users\green198800\Anaconda3\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
C:\Users\green198800\Anaconda3\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Pl

In [25]:
train_data, test_data = dataset.split()
train_data, valid_data = train_data.split()

In [26]:
MAX_VOCAB_SIZE = 300

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [27]:
train_iterator, valid_iterator, test_iterator = map(lambda e: torchtext.data.Iterator(e, batch_size=BATCH_SIZE, device=device),
                                     (train_data, valid_data, test_data))

for text, label in train_iterator:
    print(text)
    print(text.shape)
    print(label)
    print(label.shape)
    break

(tensor([[ 4],
        [ 2],
        [ 6],
        [15],
        [ 5],
        [ 9],
        [ 6],
        [ 3],
        [12],
        [23],
        [ 2],
        [21],
        [ 4],
        [ 7],
        [ 4],
        [16],
        [14],
        [ 3],
        [18],
        [10],
        [17],
        [ 9],
        [ 8],
        [ 5],
        [18],
        [ 2],
        [ 3],
        [11],
        [ 2],
        [27],
        [ 5],
        [13],
        [ 8],
        [ 8],
        [11],
        [ 3],
        [ 6],
        [ 9],
        [ 2],
        [ 4],
        [13],
        [ 3],
        [ 6],
        [ 4],
        [ 2],
        [ 2],
        [ 9],
        [11],
        [19],
        [15],
        [ 5],
        [12],
        [10],
        [10],
        [24],
        [10],
        [12],
        [ 8],
        [ 7],
        [16],
        [ 4],
        [ 9],
        [17],
        [ 8]]), tensor([[3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2.,

AttributeError: 'tuple' object has no attribute 'shape'